# Импортирование необходимых модулей

In [1]:
import re
from typing import List

import pyspark.sql as sql

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, explode, rank, desc
from pyspark.sql.functions import col, max, sum, lower, countDistinct

# Инициализация сессии

In [2]:
spark = SparkSession \
    .builder \
    .appName("L2_reports_with_apache_spark") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")\
    .getOrCreate()

In [3]:
spark.version

'3.1.2.0-eep-800'

# Загрузка данных

## Указание путей с файлами датасетов

In [4]:
import os
data_path = os.path.join(os.curdir, "data")
posts_path = os.path.join(data_path, "posts_sample.xml")
prog_lang_path = os.path.join(data_path, "programming-languages.csv")

## Чтение данных о постах Stack Overflow

In [5]:
posts_data = spark.read.format('xml').options(rowTag='row').load(posts_path)

print("Posts")
posts_data.printSchema()

Posts
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



## Чтение данных о языках программирования

In [6]:
prog_lang_data = spark.read\
.option("header", True)\
.option("inferSchema", True)\
.option("timestampFormat", 'M/d/y H:m')\
.csv(prog_lang_path)

print("Programming languages")
prog_lang_data.printSchema()

Programming languages
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



# Работа с данными

## Функции для обработки содержимого столбцов с тегами и датой последней активности поста

In [7]:
def get_tags(tags_string):
    if tags_string is None:
        return []
    
    pattern = r'<(.+?)>'
    tags = re.findall(pattern, tags_string)
    
    return tags

def get_year(date_and_time):
    return date_and_time.year

get_tags_udf = udf(get_tags, ArrayType(StringType()))
get_year_udf = udf(get_year, IntegerType())

## Выбор необходимых столбцов

In [8]:
# Применение приведенных выше функция для преобразования данных в нужный вид
posts_data_simplified = posts_data \
                        .withColumn("tags", get_tags_udf(posts_data["_Tags"])) \
                        .withColumn("year", get_year_udf(posts_data["_LastActivityDate"]))

# Выбор только нужных столбцов
posts_data_simplified = posts_data_simplified.select(
    col("tags"),
    col("year"),
    col("_ViewCount").alias("views")
)

# Отображение полученных данных
posts_data_simplified.show()

+--------------------+----+------+
|                tags|year| views|
+--------------------+----+------+
|[c#, floating-poi...|2019| 42817|
|[html, css, inter...|2019| 18214|
|                  []|2017|  null|
|[c#, .net, datetime]|2019|555183|
|[c#, datetime, ti...|2019|149445|
|                  []|2018|  null|
|[html, browser, t...|2019|176405|
|        [.net, math]|2018|123231|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2010|  null|
|                  []|2013|  null|
|                  []|2010|  null|
+--------------------+----+------+
only showing top 20 rows



## Удаление тегов, не соответствующих языкам программирования

In [9]:
# Добавление столбца, содержащего название языка программирования, но в нижнем регистре
prog_lang_data_modified = prog_lang_data.withColumn("tag_lowercase", lower(prog_lang_data["name"]))

# Разбиение массива тегов на отдельные столбцы
posts_data_filtered = posts_data_simplified.select("year", explode("tags").alias("tag"), "views")

# Объединение таблиц для того, чтобы оставить только те строки, где поле tag у постов является языком программирования
posts_data_filtered = posts_data_filtered.join(prog_lang_data_modified, 
                                    (posts_data_filtered["tag"] == prog_lang_data_modified["tag_lowercase"]), 
                                    "inner")

# Выбор нужных столбцов
posts_data_filtered = posts_data_filtered.select("year", "tag", "views")

# Отображение результата
posts_data_filtered.show()

+----+-----------+-----+
|year|        tag|views|
+----+-----------+-----+
|2010|       java|  132|
|2010|        php| 1258|
|2015|       ruby| 9649|
|2010|          c| 2384|
|2015|        php| 1987|
|2010|     python| 3321|
|2010| javascript|  128|
|2010|applescript|  477|
|2010|        php| 1748|
|2010|        php|  998|
|2013| javascript| 2095|
|2010|        sed|  447|
|2015|     python| 6558|
|2015|       java|  214|
|2015|       ruby|  214|
|2013|objective-c|  852|
|2010| javascript|  179|
|2010|          r| 6709|
|2010|        php|   78|
|2010| javascript| 1280|
+----+-----------+-----+
only showing top 20 rows



## Составление полного отчета

In [10]:
# Группировка по году последней активности и тегам, суммирование всех просмотров для каждого языка программирования в пределах одного года
posts_data_sorted = posts_data_filtered.groupBy("year", "tag").agg(sum("views").alias("total_views"))

# Сортировка по году и количеству просмотров
posts_data_sorted = posts_data_sorted.orderBy("year", desc("total_views"))

# Отображение результата
posts_data_sorted.show()

+----+------------+-----------+
|year|         tag|total_views|
+----+------------+-----------+
|2008|        java|      11532|
|2008|        ruby|       1843|
|2008|         x++|       1363|
|2009|      python|      32219|
|2009|  javascript|      17139|
|2009|           c|      16356|
|2009|        java|      13533|
|2009|         php|      12876|
|2009|        bash|       4410|
|2009|     haskell|       3992|
|2009|       xpath|       3869|
|2009| objective-c|       3671|
|2009|      delphi|       3477|
|2009|        ruby|       2844|
|2009|  powershell|        536|
|2009|actionscript|        318|
|2010|        java|      53333|
|2010|      matlab|      51865|
|2010| objective-c|      43878|
|2010|         php|      39730|
+----+------------+-----------+
only showing top 20 rows



## Составление итогового отчета (N самых популярных языков программирования за год)

In [11]:
# Разбиение по году последней активности для нумерации языков программирования в соответствии с их популярностью
window_spec = Window.partitionBy("year").orderBy(posts_data_sorted["total_views"].desc())

# Добавление столбца rank, определяющего положение языка программирования в топе
posts_data_with_ranks = posts_data_sorted.withColumn("rank", rank().over(window_spec))

# Оставляем только первые N языков программирования для каждого года
languages_per_year = 10
posts_data_sorted_result = posts_data_with_ranks.filter(posts_data_with_ranks["rank"] <= languages_per_year)

# Удаление столбца rank
posts_data_sorted_result = posts_data_sorted_result.drop(col("rank"))

# Сортировка по году и количеству просмотров
posts_data_sorted_result = posts_data_sorted_result.orderBy("year", desc("total_views"))

# Отображение результата
posts_data_sorted_result.show()

+----+-----------+-----------+
|year|        tag|total_views|
+----+-----------+-----------+
|2008|       java|      11532|
|2008|       ruby|       1843|
|2008|        x++|       1363|
|2009|     python|      32219|
|2009| javascript|      17139|
|2009|          c|      16356|
|2009|       java|      13533|
|2009|        php|      12876|
|2009|       bash|       4410|
|2009|    haskell|       3992|
|2009|      xpath|       3869|
|2009|objective-c|       3671|
|2009|     delphi|       3477|
|2010|       java|      53333|
|2010|     matlab|      51865|
|2010|objective-c|      43878|
|2010|        php|      39730|
|2010| javascript|      37059|
|2010|     python|      25930|
|2010|       ruby|      15864|
+----+-----------+-----------+
only showing top 20 rows



## Сохранение результата в формате Apache Parquet

In [12]:
posts_data_sorted_result.write.parquet("/user/sergei_sgibnev/output")

In [13]:
!hadoop fs -ls /user/sergei_sgibnev

Found 4 items
drwxr-xr-x   - sergei_sgibnev sergei_sgibnev          3 2023-11-07 13:05 /user/sergei_sgibnev/.sparkStaging
drwxr-xr-x   - sergei_sgibnev sergei_sgibnev          6 2023-11-06 14:14 /user/sergei_sgibnev/data
drwxr-xr-x   - sergei_sgibnev sergei_sgibnev        114 2023-11-07 13:07 /user/sergei_sgibnev/output
drwxr-xr-x   - sergei_sgibnev sergei_sgibnev          0 2023-11-03 17:46 /user/sergei_sgibnev/spark-warehouse


In [14]:
!hadoop fs -get /user/sergei_sgibnev/output output